In [1]:
# import all from imports.py
from imports import *

# # !!! Warning !!! 
# # *** no more warnings ***
import warnings
warnings.filterwarnings("ignore")

# custom mods
from tidy import *
from wrangle import *
from model import *

In [2]:
df = wrangle_df()

cached csv


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hinzlehome/codeup-data-science/financial_forecaster/project_csvs/avg-fees-per-transaction.csv'

In [ ]:
df.head()

In [ ]:
def add_df_circ(df):
    '''
    
    '''
    df_circ = pd.read_csv('circulation_btc.csv')
    df_circ['Timestamp'] = pd.to_datetime(df_circ['Timestamp'])
    df_circ = df_circ.set_index('Timestamp')
    df_circ = df_circ.resample('D').mean()
    df = df.join(df_circ)
    
    df['flow'] = df['total-bitcoins'] - df['total-bitcoins'].shift(1)
    # Add a new column that is the daily df.total-bitcoins to df.flow ratio
    df['stock_flow_ratio'] = df['flow'] / df['total-bitcoins']
    # Column for if stock_flow_ratio of today went up or down from yesterday
    df['stock_flow_ratio_change'] = df['stock_flow_ratio'].shift(1) - df['stock_flow_ratio']
    return df

In [ ]:
df = add_df_circ(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.dropna()

In [ ]:
df.info()

In [ ]:
df.close.sum() - df.open.sum()

In [ ]:
# Splits
def split_i(df):
    train = df.loc[:'2022-3-22']
    validate =df.loc['2022-03-23':'2022-04-23'] 
    return train, validate

def split_ii(df):
    train = df.loc[:'2022-04-24']
    test = df.loc['2022-04-25':]
    return train, test

In [ ]:
# Scaling our continuous variables
quants = [
        'open', 'high', 'low', 'close', 'volume', 'avg-fees-per-transaction', 'cost-per-transaction-percent', 
        'cost-per-transaction', 'difficulty', 'hash-rate', 'miners-revenue', 'transaction-fees-to-miners',
        'total-bitcoins', 'flow', 'stock_flow_ratio', 'stock_flow_ratio_change', 'sentiment'
         ]
scaler = MinMaxScaler()


In [ ]:
train_i, validate_i = split_i(df)
train_ii, test_ii = split_ii(df)

In [ ]:
# Scaling for the initial train/validate split

scaler.fit(train_i[quants])

train_i[quants] = scaler.transform(train_i[quants])
validate_i[quants] = scaler.transform(validate_i[quants])

In [ ]:
train_i.head()

In [ ]:
# Dropping our target and all fwd columns (to avoid 'looking' into the future)
x_train_i = train_i.drop(columns=['fwd_log_ret', 'fwd_close_positive'])
x_train_ii = train_ii.drop(columns=['fwd_log_ret', 'fwd_close_positive'])

y_train_i = train_i['fwd_log_ret']
y_train_ii = train_ii['fwd_log_ret']

x_validate_i = validate_i.drop(columns=['fwd_log_ret', 'fwd_close_positive'])
x_test_ii = test_ii.drop(columns=['fwd_log_ret', 'fwd_close_positive'])

y_validate_i = validate_i['fwd_log_ret']
y_test_ii = test_ii['fwd_log_ret']

In [ ]:
y_train_i = pd.DataFrame(y_train_i)
y_train_ii = pd.DataFrame(y_train_ii)

y_validate_i = pd.DataFrame(y_validate_i)
y_test_ii = pd.DataFrame(y_test_ii)

In [ ]:
# Testing feature set 
target = 'fwd_log_ret'
features1 = ['open', 'high', 'low', 'close', 'volume', 'sentiment', 'stock_flow_ratio']

In [ ]:
def ols_model(x_train, y_train, x_validate, y_validate, features, target):
    '''
    This function takes in our x and y components for train and validate and prints the results of RMSE for train and validate modeling.
    '''
    # Creation and fitting of the model
    lm = LinearRegression(normalize=True)
    lm.fit(x_train[features], y_train[target])
    # Prediction creation and RMSE value for train
    y_train['ret_lm_pred'] = lm.predict(x_train[features])
    rmse_train = round(mean_squared_error(y_train[target], y_train.ret_lm_pred)**(1/2), 6)
    # Prediction creation and RMSE for validate
    y_validate['ret_lm_pred'] = lm.predict(x_validate[features])
    rmse_validate = round(mean_squared_error(y_validate[target], y_validate.ret_lm_pred)**(1/2), 6)
    # Outputing the RMSE values
    print("RMSE for OLS using LinearRegression\nTraining: ", rmse_train,
            "\nValidation: ", rmse_validate)

In [ ]:
ols_model(x_train_i, y_train_i, x_validate_i, y_validate_i, features1, target)

In [ ]:
features2 = ['open', 'close', 'volume', 'sentiment', 'stock_flow_ratio' ]

In [ ]:
ols_model(x_train_i, y_train_i, x_validate_i, y_validate_i, features2, target)

In [ ]:
features3 = ['open', 'close', 'sentiment', 'stock_flow_ratio', 'cross', 'histy']

In [ ]:
ols_model(x_train_i, y_train_i, x_validate_i, y_validate_i, features3, target)

In [ ]:
df.stock_flow_ratio.plot()

In [ ]:
df3 = wrangle_3_df()

In [ ]:
df3.head()

In [ ]:
df3.info()